In [ ]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO

In [9]:
import pandas as pd
from pathlib import Path


def read_correct_sheet(file):
    xls = pd.ExcelFile(file)

    # Pick Zeitreihen sheet (Swissgrid time series)
    for name in xls.sheet_names:
        n = name.lower().replace(" ", "")
        if "zeitreihen" in n:
            return pd.read_excel(file, sheet_name=name, header=0)

    # Fallback: first sheet
    return pd.read_excel(file, sheet_name=0, header=0)


def merge_swissgrid_excels(folder, output_file):
    folder = Path(folder)

    # Read both .xls and .xlsx
    files = sorted(list(folder.glob("*.xls")) + list(folder.glob("*.xlsx")))

    if not files:
        raise ValueError("No Excel files found")

    print("Found", len(files), "files")

    # First file defines column structure
    base = read_correct_sheet(files[0])
    cols = base.columns

    frames = [base]

    for f in files[1:]:
        print("Reading", f.name)
        df = read_correct_sheet(f)
        df = df.reindex(columns=cols)
        frames.append(df)

    merged = pd.concat(frames, ignore_index=True)

    Path(output_file).parent.mkdir(parents=True, exist_ok=True)
    merged.to_csv(output_file, index=False)

    print("Saved ->", output_file)


In [10]:
merge_swissgrid_excels(
    folder="../data/raw",   # where Excel files are
    output_file="../data/processed/swissgrid_all_years.csv"
)


Found 18 files
Reading EnergieUebersichtCH-2010.xls
Reading EnergieUebersichtCH-2011.xls
Reading EnergieUebersichtCH-2012.xls
Reading EnergieUebersichtCH-2013.xls
Reading EnergieUebersichtCH-2014.xls
Reading EnergieUebersichtCH-2015.xls
Reading EnergieUebersichtCH-2016.xls
Reading EnergieUebersichtCH-2017.xls
Reading EnergieUebersichtCH-2018.xls
Reading EnergieUebersichtCH-2019.xls
Reading EnergieUebersichtCH-2020.xlsx
Reading EnergieUebersichtCH-2021.xlsx
Reading EnergieUebersichtCH-2022.xlsx
Reading EnergieUebersichtCH-2023.xlsx
Reading EnergieUebersichtCH-2024.xlsx
Reading EnergieUebersichtCH-2025.xlsx
Reading EnergieUebersichtCH-2026.xlsx
Saved -> ../data/processed/swissgrid_all_years.csv


In [11]:
import pandas as pd
import requests
from io import BytesIO
from pathlib import Path


SWISSGRID_URL_TEMPLATE = (
    "https://www.swissgrid.ch/content/dam/dataimport/energy-statistic/"
    "EnergieUebersichtCH-{year}.{ext}"
)


def _coerce_datetime_column(df):
    candidates = [df.columns[0]]

    for col in df.columns:
        lower = str(col).lower()
        if "date" in lower or "time" in lower or lower.startswith("unnamed"):
            if col not in candidates:
                candidates.append(col)

    best_col = None
    best_parsed = None
    best_ratio = -1.0

    for col in candidates:
        parsed = pd.to_datetime(df[col], errors="coerce")
        ratio = parsed.notna().mean()
        if ratio > best_ratio:
            best_ratio = ratio
            best_col = col
            best_parsed = parsed

    if best_col is None or best_ratio < 0.95:
        raise ValueError("Could not reliably identify datetime column")

    return best_col, best_parsed


def fetch_and_append_year(year, output_file):
    output_file = Path(output_file)

    if not output_file.exists():
        raise ValueError("Existing CSV not found. Run merge first.")

    # 1) Download file
    content = None

    for ext in ["xlsx", "xls"]:
        url = SWISSGRID_URL_TEMPLATE.format(year=year, ext=ext)
        r = requests.get(url, timeout=90)

        if r.status_code == 404:
            continue

        r.raise_for_status()
        if r.content:
            content = r.content
            break

    if content is None:
        raise ValueError(f"No file available for {year}")

    # 2) Read sheet
    df_new = read_correct_sheet(BytesIO(content))

    # 3) Load historical
    hist = pd.read_csv(output_file, low_memory=False)

    # Align columns strictly
    df_new = df_new.reindex(columns=hist.columns)

    # 4) Concatenate
    merged = pd.concat([hist, df_new], ignore_index=True)

    # 5) Parse datetime and deduplicate
    dt_col, dt_values = _coerce_datetime_column(merged)
    merged[dt_col] = dt_values
    merged = merged.dropna(subset=[dt_col])
    merged = merged.sort_values(dt_col)
    merged = merged.drop_duplicates(subset=[dt_col], keep="last")
    merged = merged.reset_index(drop=True)

    # 6) Save
    merged.to_csv(output_file, index=False)

    print(f"{year} appended successfully. Rows: {len(merged):,}")
    return merged


df = fetch_and_append_year(2026, "../data/processed/swissgrid_all_years.csv")


C:\Users\alvar\AppData\Local\Temp\ipykernel_33048\1285177842.py:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(df[col], errors="coerce")


2026 appended successfully. Rows: 598,971


In [12]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import time


# City coordinates
CITIES = {
    "zurich": (47.3769, 8.5417),
    "geneva": (46.2044, 6.1432),
    "basel": (47.5596, 7.5886),
    "bern": (46.9480, 7.4474),
    "lausanne": (46.5197, 6.6323),
    "lugano": (46.0037, 8.9511),
}

# Approximate population weights
WEIGHTS = {
    "zurich": 0.30,
    "geneva": 0.18,
    "basel": 0.15,
    "bern": 0.15,
    "lausanne": 0.12,
    "lugano": 0.10,
}

NASA_URL = (
    "https://power.larc.nasa.gov/api/temporal/hourly/point"
    "?parameters=T2M"
    "&community=RE"
    "&longitude={lon}"
    "&latitude={lat}"
    "&start={start}"
    "&end={end}"
    "&format=JSON"
)


def _fetch_json_with_retry(url, timeout=60, retries=3, sleep_seconds=2):
    last_err = None
    for attempt in range(1, retries + 1):
        try:
            r = requests.get(url, timeout=timeout)
            r.raise_for_status()
            return r.json()
        except requests.RequestException as err:
            last_err = err
            if attempt < retries:
                time.sleep(sleep_seconds)
    raise RuntimeError(f"Request failed after {retries} attempts: {url}") from last_err


def fetch_weather(start_year=2009):
    end_year = datetime.now().year
    all_cities_data = []

    for city, (lat, lon) in CITIES.items():
        print("Fetching:", city)
        city_frames = []

        for year in range(start_year, end_year + 1):
            start = f"{year}0101"
            end = f"{year}1231"

            # For current year, stop at yesterday to avoid partial current-day data
            if year == end_year:
                end_date = datetime.now() - timedelta(days=1)
                if end_date.year < year:
                    continue
                end = end_date.strftime("%Y%m%d")

            url = NASA_URL.format(lon=lon, lat=lat, start=start, end=end)
            json_data = _fetch_json_with_retry(url)

            if "properties" not in json_data or "parameter" not in json_data["properties"]:
                print("No data for:", city, year)
                continue

            data = json_data["properties"]["parameter"].get("T2M", {})
            if not data:
                print("No T2M data for:", city, year)
                continue

            df_year = pd.DataFrame({
                "datetime": pd.to_datetime(list(data.keys()), format="%Y%m%d%H"),
                city: list(data.values()),
            })

            df_year[city] = df_year[city].replace(-999, pd.NA)
            city_frames.append(df_year)

        if not city_frames:
            raise ValueError(f"No data fetched for {city}")

        df_city = pd.concat(city_frames, ignore_index=True)
        all_cities_data.append(df_city)

    # Merge city time series
    df_weather = all_cities_data[0]
    for df in all_cities_data[1:]:
        df_weather = df_weather.merge(df, on="datetime", how="inner")

    # Clean
    for city in CITIES:
        df_weather[city] = pd.to_numeric(df_weather[city], errors="coerce")

    df_weather = df_weather.dropna()

    # Weighted temperature and derived features
    df_weather["temp_weighted"] = sum(df_weather[c] * WEIGHTS[c] for c in CITIES)
    df_weather["HDH"] = (18 - df_weather["temp_weighted"]).clip(lower=0)
    df_weather["CDH"] = (df_weather["temp_weighted"] - 22).clip(lower=0)
    df_weather["temp_72h"] = df_weather["temp_weighted"].rolling(72, min_periods=1).mean()
    df_weather["extreme_cold"] = (df_weather["temp_weighted"] < -5).astype(int)

    return df_weather.reset_index(drop=True)


In [13]:
df_weather = fetch_weather()

Fetching: zurich
Fetching: geneva
Fetching: basel
Fetching: bern
Fetching: lausanne
Fetching: lugano


In [14]:
df_weather

,datetime,zurich,geneva,basel,bern,lausanne,lugano,temp_weighted,HDH,CDH,temp_72h,extreme_cold
0,2009-01-01 00:00:00,-1.60,-1.11,-3.80,-3.13,-3.89,-5.22,-2.7081,20.7081,0.0,-2.708100,0
1,2009-01-01 01:00:00,-1.60,-1.45,-4.15,-3.84,-4.68,-4.89,-2.9901,20.9901,0.0,-2.849100,0
2,2009-01-01 02:00:00,-1.96,-1.99,-4.52,-4.65,-5.43,-4.58,-3.4313,21.4313,0.0,-3.043167,0
3,2009-01-01 03:00:00,-2.43,-2.84,-4.89,-5.34,-6.25,-4.35,-3.9597,21.9597,0.0,-3.272300,0
4,2009-01-01 04:00:00,-2.89,-3.63,-5.24,-5.96,-7.23,-4.51,-4.5190,22.5190,0.0,-3.521640,0
...,...,...,...,...,...,...,...,...,...,...,...,...
150139,2026-02-19 19:00:00,3.36,-0.13,3.80,1.49,0.26,0.26,1.8353,16.1647,0.0,0.661831,0
150140,2026-02-19 20:00:00,3.37,-0.14,3.50,1.19,0.21,-0.43,1.6715,16.3285,0.0,0.679201,0
150141,2026-02-19 21:00:00,2.75,-0.16,3.39,1.11,0.17,-0.88,1.4036,16.5964,0.0,0.697281,0
150142,2026-02-19 22:00:00,2.32,-0.18,3.39,1.07,0.05,-1.35,1.2036,16.7964,0.0,0.712878,0
